# Dynamical Sampling on Graphs

In the interactive code below you can 

* Build a graph by specifying its nodes and edges: it visulaizes the graph and returns the Laplacian spectrum.

* Set the sampling locations, the number of iterations and the PW dimension, it returns the uper and lower frame bounds of the resulting iterative system. 

In [1]:
###### nbi:hide_in

# Importing needed libraries 
import matplotlib
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csgraph
from scipy.sparse.linalg import eigsh, svds
from scipy.linalg import eigh

# Creating the graph class
class Graph(object):
    r"""
    Args:
        edges ([num_edges, 3] array): Graph connectivity in COO format 
        (instead of saving the adjacency matrix coo format saves only the node 
        values so the weights need to be given separetely). Third argument is 
        the weight. 
    """
    def __init__(self,  N_nodes=1, edges=[], samples=[], **kwargs):
        self.edges = edges
        self.N_nodes = N_nodes
        self.nodes = [i for i in range(N_nodes)]
        self.samples = samples
        
    def adj(self):
        adjacency_matr = np.zeros([self.N_nodes, self.N_nodes])
        for idx, row in enumerate(self.edges):
            ind1 = self.nodes.index(row[0])
            ind2 = self.nodes.index(row[1])
            adjacency_matr[ind1, ind2] = row[2]
            adjacency_matr[ind2, ind1] = adjacency_matr[ind1, ind2]
        return adjacency_matr
        
    def add_node(self):
        self.N_nodes += 1
        self.nodes.append(max(self.nodes)+1)
            
    def add_edge(self, edge):
        if edge!=None:
            self.edges.append(edge)
            
    def add_sample(self, node):
        if node not in self.samples:
            self.samples.append(node)
            
    def del_sample(self, node):
        if node in self.samples:
            self.samples.remove(node)
            
    def del_node(self, node):
        if node in self.nodes:
            self.N_nodes-=1
            self.edges = [item for item in self.edges if item[0]!=node and item[1]!=node]
            self.nodes.remove(node)
            self.del_sample(node)
    
    def del_edge(self, pair):
        self.edges[:] = [item for item in self.edges if item[:2]!=pair and item[:2]!=(pair[1], pair[0])]
        
    def change_edge(self, newedge):
        for edge in self.edges:
            if (edge[0], edge[1])==(newedge[0], newedge[1]) or (edge[1], edge[0])==(newedge[0], newedge[1]):
                self.del_edge((newedge[0], newedge[1]))
                self.add_edge(newedge)
                       
    #reset graph
    def reset(self):
        self.N_nodes = 1
        self.nodes = [i for i in range(self.N_nodes)]
        self.edges = []
    
    def lapl_eigen(self):
        Adj = self.adj()
        Lapl = csgraph.laplacian(Adj, normed=False)
        vals, U = eigh(Lapl)
        return vals, U

In [2]:
###### nbi:hide_in
%matplotlib inline

from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import Button, Layout, GridspecLayout

# Generate the graph
G = Graph()
fig, ax = plt.subplots(figsize=(10, 5))
ax.spines["top"].set_visible(False)  
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["left"].set_visible(False)
plt.close()

# output will capture the figure and grid only
# it will be cleared every time and rewritten
output = widgets.Output()

def draw_graph(G, ax, output): 
    #create the graph
    Gnx = nx.Graph()
    Gnx.add_nodes_from(G.nodes)
    Gnx.add_weighted_edges_from(G.edges)
    
    # colors
    node_colors = ["red" if node in G.samples else "blue" for node in G.nodes]
    
    #plot
    with output:
        ax.cla()
        nx.draw_networkx(Gnx, ax=ax, node_color=node_colors)
        display(fig); 
        
# edge display
def edge_disp(G, output):
    grid = GridspecLayout(G.N_nodes+1, G.N_nodes+2, justify_items='center')
    for i in range(1, G.N_nodes+1):
        if G.N_nodes>1:
            grid[0,i+1] = widgets.Label(str(G.nodes[i-1]))
            grid[i,1] = widgets.Label(str(G.nodes[i-1]))
        for j in range(i+1, G.N_nodes+2):
            grid[i,j] = widgets.FloatText(
                value=0,
                min=0,
                max=10.0,
                step=0.1,
                disabled=False,
                display='inline-flex',
                layout={'width': '40px'},
                color="b",
                description_tooltip=str(G.nodes[i-1])+" "+str(G.nodes[j-2])
            )
            grid[j-1,i+1] = widgets.FloatText(
                value=0,
                min=0,
                max=10.0,
                step=0.1,
                disabled=False,
                display='inline-flex',
                layout={'width': '40px'},
                color="b",
                description_tooltip=str(G.nodes[i-1])+" "+str(G.nodes[j-2])
            )
            l = widgets.link((grid[j-1,i+1], 'value'), (grid[i,j], 'value'))
            
        grid[i,0] = widgets.Checkbox(
            value=False,
            description_tooltip=str(G.nodes[i-1]))
    grid[0, 0] =  widgets.Label("Samples")
    for edge in G.edges:
        ind1 = G.nodes.index(edge[0])
        ind2 = G.nodes.index(edge[1])
        i,j = sorted([ind1, ind2])
        grid[i+1,j+2].value = edge[2]
    for node in G.samples:
        i = G.nodes.index(node)
        grid[i+1, 0].value = True
        
    for i in range(1, G.N_nodes+1):
            for j in range(i+1, G.N_nodes+2):
                grid[i,j].observe(on_value_change, names='value')
            grid[i,0].observe(on_click_change, names='value');
    with output:
        display(grid) 
        
def draw_widget(G, output, show=True):
    # clear the output
    output.clear_output()
    # add the grid to output
    edge_disp(G, output)
    # add the graph to the output
    draw_graph(G, ax, output)
    # add eigenvalues to the output
    eig, U = G.lapl_eigen()
    with output:
        display("Laplacian eigenvalues are {}".format(eig))
        if show==True:
            display("Laplacian eigenvectors are {}".format(U))

def dynamic(A, L, V):
    Mat = np.eye(A.shape[0])
    for i in range(L-1):
        Mat = np.concatenate([np.eye(A.shape[0]), Mat @ A])
    F = Mat @ V
    return F.reshape(A.shape[0], L*V.shape[1], order="F")
            
def gds(G, pw_dim, L, options, output):
    # sampling matrix
    S = np.zeros([G.N_nodes, len(G.samples)])
    for j, node in enumerate(G.samples):
        i = G.nodes.index(node)
        S[i, j]=1
        
    Adj = G.adj() # the adjoint matrix
    Lap = csgraph.laplacian(Adj, normed=False)
    
    # Compute PW eigenvectors
    vals, U =  eigh(Lap, subset_by_index=[0,pw_dim-1])
                            
    # Compute the dynamical sampling vectors
    if options==0:
        B = dynamic(Lap, L, S)
    if options==1:
        B = dynamic(Adj, L, S)
                            
    # Project onto PW space
    PF = U.transpose() @ B
                            
    # Compute frame bounds
    Frame_op = PF @ PF.transpose()
    low = svds(Frame_op, k=1, which='SM', return_singular_vectors=False)[0]
    up = svds(Frame_op, k=1, which='LM', return_singular_vectors=False)[0]

    # print
    with output:
        display("Lower frame bound = {}".format(low))
        display("Upper frame bound = {}".format(up))
        
def on_value_change(change):
    if  change["new"]>0:
        edge_index =[int(num) for num in change.owner.description_tooltip.split()]
        edge = (edge_index[0],  edge_index[1], change["new"])
        if change["old"]==0:
            G.add_edge(edge)
        else:
            G.change_edge(edge)
        draw_widget(G, output, show_eigen.value)
    if change["new"]==0 and change["old"]>0:
        edge_index =[int(num) for num in change.owner.description_tooltip.split()]
        pair = (edge_index[0],  edge_index[1])
        G.del_edge(pair)
        draw_widget(G, output, show_eigen.value)
    if change["new"]<0:
        raise NameError('Weights must be non-negative')
                
def on_click_change(change):
    if change["new"]==True:
        node =int(change.owner.description_tooltip)
        G.add_sample(node)
    if change["new"]==False:
        node =int(change.owner.description_tooltip)
        G.del_sample(node)
    draw_widget(G, output, show_eigen.value)
        
# add_node button
button1 = widgets.Button(
    description='Add node',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

def button1_click(b):
    G.add_node()
    draw_widget(G, output, show_eigen.value)
        
# reset graph
button2 = widgets.Button(
    description='Reset graph',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

def button2_click(b):
    G.reset()
    draw_widget(G, output, show_eigen.value)

# remove a node
button3 = widgets.Button(
    description='Remove node',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

rmnode = widgets.IntText(
                    layout={'width': '100px'},
                )

def button3_click(b):
    G.del_node(rmnode.value)
    draw_widget(G, output, show_eigen.value)

button4 = widgets.Button(
    description='Frame bounds',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)
        
# do dynamical sampling
num_iter_widget = widgets.BoundedIntText(
                    min=1,
                    layout={'width': '100px'},
                    discription="Number of iteration"
                )
pw_dim_widget = widgets.BoundedIntText(
                    min=2,
                    layout={'width': '100px'},
                    discription="PW dim"
                )

mat_select = widgets.ToggleButtons(
    options=[('Laplacian',0), ('Adjacency',1)],
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

show_eigen = widgets.Checkbox(
            value=True,
            description="Show laplacian eigenvalues")


def button4_click(b):
    draw_widget(G, output, show_eigen.value)
    gds(G, pw_dim_widget.value, num_iter_widget.value, mat_select.value, output)
        
button1.on_click(button1_click)
button2.on_click(button2_click)
button3.on_click(button3_click)
button4.on_click(button4_click)

# to access the children of controls, do controls.children so can add new widgets
controls = widgets.VBox([widgets.HBox([button2, button1, button4]), 
                         widgets.HBox([rmnode, button3]),
                         widgets.HBox([mat_select,show_eigen]),
                         widgets.HBox([num_iter_widget, widgets.Label(" = iterations")]),
                         widgets.HBox([pw_dim_widget, widgets.Label(" = PW dimension")])])
draw_widget(G, output) # capture grid and fig with outcome 
display(controls, output);

Output()